In [3]:
import os
cwd = os.getcwd()
cwd

'/home/hannesr/dev/slo/cassandra-strict-slo/data_analysis'

In [23]:
RESULTS_DIR = "../results"
BENCHMARK = "high_delay_looptest_40000ops_"
# BENCHMARK = "low_delay_looptest_40000ops_"
PERCENTILE = 95.0
# PERCENTILE = 60.0

In [24]:
def get_mixload_for_prefix(prefix):
    mixload_files = list(map(lambda file: RESULTS_DIR + "/" + file,
                        filter(lambda file: 
                           file.startswith(prefix) and not file.endswith("stderr") and not file.endswith("metrics"),
                           os.listdir(RESULTS_DIR)
                        )))
    return mixload_files
get_mixload_for_prefix(BENCHMARK)

['../results/high_delay_looptest_40000ops_1',
 '../results/high_delay_looptest_40000ops_10',
 '../results/high_delay_looptest_40000ops_2',
 '../results/high_delay_looptest_40000ops_3',
 '../results/high_delay_looptest_40000ops_4',
 '../results/high_delay_looptest_40000ops_5',
 '../results/high_delay_looptest_40000ops_6',
 '../results/high_delay_looptest_40000ops_7',
 '../results/high_delay_looptest_40000ops_8',
 '../results/high_delay_looptest_40000ops_9']

In [45]:
def get_percentiles(filename):
    file = open(filename, "r")
    lines = list(map(lambda line: line.rstrip('\n'), file.readlines()))

    # Remove everything before "Starting test"
    def get_line_with_find(lines, regex):
        for index, line in enumerate(lines):
            if line.find(regex) != -1:
                return index
        return -1

    test_index = get_line_with_find(lines, 'Starting test.')

    full_test = lines[test_index + 1:]
    
    read_test_lines = list(
        filter(lambda line: line.find("[READ]") != -1, 
        filter(lambda line: line.find("Percentile") != -1, full_test)
    ))

    def get_tuple_from_line(line):
        parts = line.split(",")
        
        percentile_string = parts[1].strip()
        percentile = percentile_string.split("Percentile")[0].replace("th", "")
        percentile = float(percentile)
        
        latency_string = parts[2].strip()
        latency = int(latency_string)
        
        return percentile, latency
    
    return sorted(list(map(get_tuple_from_line, read_test_lines)), key=lambda x: x[0])

def get_cdf_for_file(filename, prefix):
    threshold = filename.split(prefix)[1].split("_")[0]
    return threshold, get_percentiles(filename)

def get_values_for_files(files, prefix):
    cdfs = list(map(get_cdf_for_file, files, [prefix] * len(files)))
    cdf_lists = list(map(lambda cdf: cdf[1], cdfs))
    percentile_list = list(map(lambda l: list(filter(lambda item: item[0] == PERCENTILE, l)), cdf_lists))
    values = list(map(lambda v: v[0][1] ,percentile_list))
    return values

In [64]:
bench_no = "no_delay_looptest_40000ops_"
bench_low = "low_delay_looptest_40000ops_"
bench_high = "high_delay_looptest_40000ops_"


a = get_values_for_files(get_mixload_for_prefix(bench_no), bench_no)
b = get_values_for_files(get_mixload_for_prefix(bench_low), bench_low)
c = get_values_for_files(get_mixload_for_prefix(bench_high), bench_high)


for l in [str(e)[1:-1] for e in list(zip(a,b,c))]:
    print(l)


7871, 8919, 25375
6215, 7111, 25167
5595, 6491, 26671
8199, 7567, 26751
6827, 8391, 24703
6971, 7939, 26655
4975, 7555, 27055
3831, 7099, 24175
8991, 7571, 27359
8663, 8727, 26095


In [65]:
bench_no = "no_delay_looptest_"
bench_low = "low_delay_looptest_"
bench_high = "high_delay_looptest_"

def filter_40000(files):
    return list(filter(lambda filename: "40000" not in filename, files))

a = get_values_for_files(filter_40000(get_mixload_for_prefix(bench_no)), bench_no)
b = get_values_for_files(filter_40000(get_mixload_for_prefix(bench_low)), bench_low)
c = get_values_for_files(filter_40000(get_mixload_for_prefix(bench_high)), bench_high)

    
for l in [str(e)[1:-1] for e in list(zip(a,b,c))]:
    print(l)


19983, 15119, 25039
15135, 14887, 24895
13367, 18223, 25183
15455, 14423, 25151
21807, 14231, 25263
14383, 15063, 25215
14199, 16959, 25183
18431, 14879, 25151
14655, 14391, 25039
14063, 18415, 25103


# Calculating average throughput

In [92]:
import statistics

def get_line_with_find(lines, regex):
    for index, line in enumerate(lines):
        if line.find(regex) != -1:
            return index
    return -1

def get_avg_throughput(filename):
    file = open(filename, "r")
    lines = list(map(lambda line: line.rstrip('\n'), file.readlines()))
    
    ops_lines = [line for line in lines if line.find("[OVERALL], Throughput(ops/sec),") != -1]
    ops_lines = [float(line.split(",")[2].strip()) for line in ops_lines]
    
    return ops_lines


def get_benchmark_info(files):
    return [v for f in files for v in get_avg_throughput(f)]


In [93]:
bench_no = "no_delay_looptest_"
bench_low = "low_delay_looptest_"
bench_high = "high_delay_looptest_"

def filter_40000(files):
    return list(filter(lambda filename: "40000" not in filename, files))

a = get_benchmark_info(filter_40000(get_mixload_for_prefix(bench_no)))
b = get_benchmark_info(filter_40000(get_mixload_for_prefix(bench_low)))
c = get_benchmark_info(filter_40000(get_mixload_for_prefix(bench_high)))

def summary(v):
    print(min(v))
    print(max(v))
    print(statistics.mean(v))
    
summary(a)
summary(b)
summary(c)

4960.784994617548
7508.578550994511
6368.966459902762
5379.699167222569
6415.2734831085845
6079.235066879465
1425.9700517769725
1437.109914477589
1430.5345253204412


In [94]:
bench_no = "no_delay_looptest_40000ops_"
bench_low = "low_delay_looptest_40000ops_"
bench_high = "high_delay_looptest_40000ops_"

a = get_benchmark_info(get_mixload_for_prefix(bench_no))
b = get_benchmark_info(get_mixload_for_prefix(bench_low))
c = get_benchmark_info(get_mixload_for_prefix(bench_high))

def summary(v):
    print(min(v))
    print(max(v))
    print(statistics.mean(v))
    
summary(a)
summary(b)
summary(c)

986.3343377504673
988.3351741075951
987.1532089395052
986.0644442417534
988.2277370820105
987.3671223741133
985.1513069263525
987.9055661069358
986.447955978116


# Extracting data from metadata (deprecated)

In [15]:
get_values_for_files(mixload_files)

[21551, 21471, 22511, 22511, 21967, 22463, 22495, 21519, 22367, 22063]

In [87]:
mixload_metrics =   list(
                    map(lambda directory: 
                        list(
                        map(lambda file: directory + "_metrics/" + file,
                            filter(lambda file: 
                               file.startswith(BENCHMARK) and not file.endswith("stderr") and not file.endswith("metrics"),
                            os.listdir(directory + "_metrics")))),
                    mixload_files))

mixload_metrics = [elem for bench in mixload_metrics for elem in bench]
mixload_metrics

mixload_info = [file + "_stderr" for file in mixload_files]
mixload_info

['../results/high_delay_looptest_40000ops_1_stderr',
 '../results/high_delay_looptest_40000ops_10_stderr',
 '../results/high_delay_looptest_40000ops_2_stderr',
 '../results/high_delay_looptest_40000ops_3_stderr',
 '../results/high_delay_looptest_40000ops_4_stderr',
 '../results/high_delay_looptest_40000ops_5_stderr',
 '../results/high_delay_looptest_40000ops_6_stderr',
 '../results/high_delay_looptest_40000ops_7_stderr',
 '../results/high_delay_looptest_40000ops_8_stderr',
 '../results/high_delay_looptest_40000ops_9_stderr']

In [88]:
RANGE = range(1000, 1000, 1)

def get_line_with_find(lines, regex):
    for index, line in enumerate(lines):
        if line.find(regex) != -1:
            return index
    return -1

def get_percentile_data(percentile, filename):
    file = open(filename, "r")
    lines = list(map(lambda line: line.rstrip('\n'), file.readlines()))
    
    ops_lines = [line for line in lines if line.find("current ops/sec;") != -1]
    ops_lines = [line.split("[") for line in ops_lines]
    
    throughput_lines = [line[0] for line in ops_lines]
    ops_lines = [line[1:] for line in ops_lines]
    
    # 5 = 90, 6 = 99
    if percentile == "90":
        index = 5
    elif percentile == "99":
        index = 6
    else:
        raise Exception("Not valid percentile")
        
    ops_lines = [(line[0].split(",")[index], line[1].split(",")[index]) for line in ops_lines]
    ops_lines = [(read.split("=")[1], update.split("=")[1]) for (read, update) in ops_lines]
    
    
    throughput_lines = [line.split(";")[1] for line in throughput_lines]
    throughput_lines = [line.split()[0] for line in throughput_lines]
    
    return list(zip(throughput_lines, ops_lines))

def get_latencies(filename):
    throughput_latency_list = get_percentile_data('90', filename)
    parsed_list = [(float(throughput), int(read), int(write)) for (throughput, (read, write)) in throughput_latency_list]
    return parsed_list

def get_latencies_for_threshold_2(threshold = 1, filename_prefix = BENCHMARK):
    lists = map(lambda load: get_latencies(load, threshold, filename_prefix), RANGE)
    all_values = list()
    for l in lists:
        all_values.extend(l)
        
    all_values = filter(lambda e: e[1]<40000, all_values)
    return list(all_values)


# get_percentile_data('99', "../results/phase3_final_400000ops_1_2500")
get_latencies(mixload_info[0])

# all_read = [e for file in mixload_info for e in get_latencies(file)]
# all_read = [r for _l, r, _w, in all_read]
# all_read

all_read_h = [e for file in mixload_info for e in get_latencies(file)]
all_read_h = [r for _l, r, _w, in all_read_h]
all_read_h

[60191,
 38079,
 35487,
 39775,
 31759,
 36063,
 32991,
 35583,
 30271,
 32591,
 33503,
 33727,
 33183,
 31375,
 30479,
 32143,
 33343,
 30063,
 29359,
 33471,
 30751,
 30351,
 30623,
 27839,
 32239,
 30879,
 28303,
 30239,
 27839,
 31359,
 29871,
 28847,
 28991,
 28239,
 29103,
 28591,
 27999,
 28367,
 29359,
 29455,
 35935,
 55455,
 37023,
 45631,
 35391,
 32959,
 35295,
 34879,
 35039,
 32767,
 30527,
 36031,
 32927,
 32159,
 29887,
 31935,
 29535,
 31375,
 29679,
 30879,
 28079,
 32767,
 30239,
 29231,
 29727,
 30591,
 28239,
 28799,
 28639,
 29887,
 31135,
 28303,
 28479,
 29503,
 29103,
 29535,
 28479,
 28751,
 28879,
 29023,
 27663,
 31775,
 49727,
 39103,
 36255,
 35423,
 33887,
 35007,
 36799,
 33215,
 34335,
 32287,
 33983,
 33503,
 33183,
 32287,
 31295,
 32575,
 30255,
 30319,
 31087,
 33439,
 31471,
 30367,
 29951,
 29503,
 31311,
 29871,
 29503,
 30239,
 30959,
 32959,
 31247,
 32015,
 31167,
 31087,
 30415,
 31567,
 32319,
 30351,
 30351,
 30303,
 33695,
 51807,
 35871,


In [89]:
for l in zip(all_read, all_read_h):
    print(str(l)[1:-1])

60831, 60191
24815, 38079
16863, 35487
15143, 39775
15799, 31759
20431, 36063
15991, 32991
15511, 35583
13487, 30271
16959, 32591
14895, 33503
16375, 33727
15607, 33183
13895, 31375
13863, 30479
14519, 32143
12423, 33343
12791, 30063
14463, 29359
15487, 33471
13063, 30751
12791, 30351
16023, 30623
12079, 27839
12239, 32239
13439, 30879
14239, 28303
15663, 30239
13055, 27839
16071, 31359
12983, 29871
14671, 28847
12327, 28991
12759, 28239
11575, 29103
13647, 28591
12775, 27999
13423, 28367
12607, 29359
12503, 29455
14271, 35935
60223, 55455
14767, 37023
20767, 45631
15863, 35391
13391, 32959
14239, 35295
14119, 34879
12447, 35039
11863, 32767
12807, 30527
13175, 36031
11599, 32927
12663, 32159
11063, 29887
12607, 31935
11791, 29535
12751, 31375
11159, 29679
10415, 30879
9743, 28079
10871, 32767
11119, 30239
11439, 29231
11271, 29727
10839, 30591
11631, 28239
10319, 28799
10231, 28639
10255, 29887
10599, 31135
9759, 28303
9183, 28479
9967, 29503
11607, 29103
11503, 29535
12903, 28479
958